# 📊 Matrix Factorization using SVD

**Objective**: Build a recommender system using Singular Value Decomposition (SVD) and explore dimensionality reduction for improving performance.

In [ ]:
# 📦 Import Required Libraries
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 📥 Load MovieLens 100K dataset from Surprise
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
trainset

In [ ]:
# 🧠 Train SVD Model
model = SVD(n_factors=50, biased=True, verbose=True)
model.fit(trainset)
predictions = model.test(testset)

In [ ]:
# 📈 Evaluate Performance
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

In [ ]:
# ⭐ Generate Top-N Recommendations for a Sample User
from collections import defaultdict

def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=5)
sample_user = list(top_n.keys())[0]
print(f"Top 5 recommendations for User {sample_user}: {top_n[sample_user]}")

In [ ]:
# 📉 Test SVD with different number of latent factors
results = []
for k in [10, 20, 50, 100]:
    model = SVD(n_factors=k)
    model.fit(trainset)
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions, verbose=False)
    results.append((k, rmse))

results_df = pd.DataFrame(results, columns=["n_factors", "RMSE"])
sns.lineplot(x='n_factors', y='RMSE', data=results_df).set(title='RMSE vs Number of Latent Factors')